[1] 표지판 모델 이미지 등록 \
[2] 도로 영상 선택 \
[3] SIFT로 표지판 ↔ 도로 영상 매칭 \
[4] 가장 매칭이 좋은 표지판 찾기 \
[5] 매칭 성공 → 위치 표시 + 안내문 + 경고음 \

In [1]:
import cv2 as cv
import numpy as np
from PyQt5.QtWidgets import *
import sys
import winsound

# GUI 윈도우 클래스 정의
class TrafficWeak(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('교통약자 보호')
        self.setGeometry(200, 200, 700, 200)

        # [1] 버튼들 만들기
        signButton = QPushButton('표지판 등록', self)
        roadButton = QPushButton('도로 영상 불러옴', self)
        recognitionButton = QPushButton('인식', self)
        quitButton = QPushButton('나가기', self)
        self.label = QLabel('환영합니다!', self)

        # [2] 버튼 배치
        signButton.setGeometry(10, 10, 100, 30)
        roadButton.setGeometry(110, 10, 100, 30)
        recognitionButton.setGeometry(210, 10, 100, 30)
        quitButton.setGeometry(510, 10, 100, 30)
        self.label.setGeometry(10, 40, 600, 170)

        # [3] 버튼 클릭 시 함수 연결
        signButton.clicked.connect(self.signFunction)
        roadButton.clicked.connect(self.roadFunction)
        recognitionButton.clicked.connect(self.recognitionFunction)
        quitButton.clicked.connect(self.quitFunction)

        # [4] 표지판 이미지 파일 이름 & 라벨
        self.signFiles = [
            ['child.png', '어린이'],
            ['elder.png', '노인'],
            ['disabled.png', '장애인']
        ]

        self.signImgs = []  # 표지판 이미지 불러와서 저장
        self.roadImg = None  # 도로 영상

    # [5] 표지판 이미지 등록
    def signFunction(self):
        self.label.clear()
        self.label.setText('교통약자 번호판을 등록합니다.')

        for fname, _ in self.signFiles:
            self.signImgs.append(cv.imread(fname))  # 이미지 읽기
            cv.imshow(fname, self.signImgs[-1])    # 각 표지판 보여주기

    # [6] 도로 영상 불러오기
    def roadFunction(self):
        if self.signImgs == []:
            self.label.setText('먼저 번호판을 등록하세요.')
        else:
            fname = QFileDialog.getOpenFileName(self, '파일 읽기', './')
            self.roadImg = cv.imread(fname[0])
            if self.roadImg is None:
                sys.exit('파일을 찾을 수 없습니다.')

            cv.imshow('Road scene', self.roadImg)

    # [7] 표지판 인식
    def recognitionFunction(self):
        if self.roadImg is None:
            self.label.setText('먼저 도로 영상을 입력하세요.')
        else:
            sift = cv.SIFT_create()  # SIFT 객체 생성

            KD = []  # 각 표지판 이미지의 키포인트와 디스크립터 저장
            for img in self.signImgs:
                gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
                KD.append(sift.detectAndCompute(gray, None))

            # 도로 영상 키포인트와 디스크립터
            grayRoad = cv.cvtColor(self.roadImg, cv.COLOR_BGR2GRAY)
            road_kp, road_des = sift.detectAndCompute(grayRoad, None)

            # FLANN 매칭기 생성
            matcher = cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED)
            GM = []  # 각 표지판에 대한 좋은 매칭 저장

            for sign_kp, sign_des in KD:
                knn_match = matcher.knnMatch(sign_des, road_des, 2)  # KNN 매칭
                T = 0.7  # ratio test 임계값
                good_match = []
                for nearest1, nearest2 in knn_match:
                    if (nearest1.distance / nearest2.distance) < T:
                        good_match.append(nearest1)
                GM.append(good_match)  # 매칭 결과 저장

            # 가장 매칭 쌍이 많은 표지판 찾기
            best = GM.index(max(GM, key=len))

            if len(GM[best]) < 4:
                self.label.setText('표지판이 없습니다.')  # 매칭 쌍 너무 적으면 실패
            else:
                # 호모그래피로 위치 찾아서 그리기
                sign_kp = KD[best][0]
                good_match = GM[best]

                # 매칭된 포인트 쌍
                points1 = np.float32([sign_kp[gm.queryIdx].pt for gm in good_match])
                points2 = np.float32([road_kp[gm.trainIdx].pt for gm in good_match])

                # 호모그래피 추정
                H, _ = cv.findHomography(points1, points2, cv.RANSAC)

                h1, w1 = self.signImgs[best].shape[:2]  # 표지판 원본 크기
                h2, w2 = self.roadImg.shape[:2]         # 도로 영상 크기

                # 표지판 경계 사각형 좌표
                box1 = np.float32([
                    [0, 0], [0, h1 - 1], [w1 - 1, h1 - 1], [w1 - 1, 0]
                ]).reshape(4, 1, 2)

                # 도로 영상 상의 대응 좌표
                box2 = cv.perspectiveTransform(box1, H)

                # 도로 영상에 표시
                self.roadImg = cv.polylines(self.roadImg, [np.int32(box2)], True, (0, 255, 0), 4)

                # 매칭 결과 시각화
                img_match = np.empty((max(h1, h2), w1 + w2, 3), dtype=np.uint8)
                cv.drawMatches(self.signImgs[best], sign_kp, self.roadImg, road_kp,
                               good_match, img_match, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
                cv.imshow('Matches and Homography', img_match)

                # 보호구역 알림 & 경고음
                self.label.setText(self.signFiles[best][1] + ' 보호구역입니다. 30km로 서행하세요.')
                winsound.Beep(3000, 500)  # 경고음

    # [8] 나가기 버튼
    def quitFunction(self):
        cv.destroyAllWindows()
        self.close()

# [9] PyQt5 앱 실행
app = QApplication(sys.argv)
win = TrafficWeak()
win.show()
app.exec_()


0


# recognitionFunction 함수 상세 흐름 정리

## 목적

등록한 표지판 이미지(어린이, 노인, 장애인)를 기준으로  
도로 영상에서 해당 표지판을 찾아 위치를 표시하고  
경고 메시지와 소리를 출력한다.

---

## 전체 단계 요약

1. 도로 영상 준비 여부 확인  
2. SIFT로 표지판 이미지들의 특징점 & 디스크립터 추출  
3. 도로 영상에서도 SIFT 특징점 & 디스크립터 추출  
4. FLANN 매칭기 + Ratio Test로 특징점 매칭  
5. 가장 매칭 쌍이 많은 표지판 선택  
6. 매칭 쌍이 충분하면 Homography로 위치 추정  
7. 도로 영상에 경계선 표시  
8. 매칭 결과 시각화  
9. 보호구역 알림 + 경고음 발생

---

## 단계별 흐름

### 1️ 도로 영상 준비 여부 확인

```python
if self.roadImg is None:
    self.label.setText('먼저 도로 영상을 입력하세요.')
```

- 도로 영상이 없으면 인식할 수 없으므로 안내 메시지를 띄운다.

---

### 2️ SIFT 객체 생성

```python
sift = cv.SIFT_create()
```

- SIFT 알고리즘으로 특징점과 디스크립터를 추출할 준비를 한다.

---

### 3️ 표지판 이미지 특징점 & 디스크립터 추출

```python
KD = []
for img in self.signImgs:
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    KD.append(sift.detectAndCompute(gray, None))
```

- 등록된 표지판 이미지를 Grayscale로 변환한다.
- SIFT로 특징점과 디스크립터를 추출하여 `KD` 리스트에 저장한다.

---

### 4️ 도로 영상 특징점 & 디스크립터 추출

```python
grayRoad = cv.cvtColor(self.roadImg, cv.COLOR_BGR2GRAY)
road_kp, road_des = sift.detectAndCompute(grayRoad, None)
```

- 도로 영상도 Grayscale로 변환 후 SIFT로 특징점과 디스크립터를 추출한다.

---

### 5️ FLANN 매칭기 + Ratio Test로 특징점 매칭

```python
matcher = cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED)
GM = []

for sign_kp, sign_des in KD:
    knn_match = matcher.knnMatch(sign_des, road_des, 2)
    T = 0.7
    good_match = []
    for nearest1, nearest2 in knn_match:
        if (nearest1.distance / nearest2.distance) < T:
            good_match.append(nearest1)
    GM.append(good_match)
```

- FLANN 매칭기로 표지판 이미지와 도로 영상의 특징점을 비교한다.
- KNN으로 두 개씩 최근접 매칭을 찾고,
- Ratio Test로 유사도가 낮은 잘못된 매칭은 제거한다.
- 좋은 매칭만 `GM`에 저장한다.

---

### 6️ 가장 매칭 쌍이 많은 표지판 선택

```python
best = GM.index(max(GM, key=len))
```

- 가장 많은 매칭 쌍을 가진 표지판을 선택한다.

---

### 7️ 매칭 쌍이 너무 적으면 실패 처리

```python
if len(GM[best]) < 4:
    self.label.setText('표지판이 없습니다.')
```

- 매칭 쌍이 4개 미만이면 Homography 계산이 불가능하므로 인식을 실패 처리한다.

---

### 8️ Homography로 위치 추정

```python
sign_kp = KD[best][0]
good_match = GM[best]

points1 = np.float32([sign_kp[gm.queryIdx].pt for gm in good_match])
points2 = np.float32([road_kp[gm.trainIdx].pt for gm in good_match])

H, _ = cv.findHomography(points1, points2, cv.RANSAC)
```

- 선택된 표지판의 매칭된 특징점 좌표 쌍을 준비한다.
- RANSAC으로 Homography 행렬을 추정한다.

---

### 9️ 표지판 경계 사각형을 도로 영상에 표시

```python
h1, w1 = self.signImgs[best].shape[:2]

box1 = np.float32([
    [0, 0], [0, h1 - 1], [w1 - 1, h1 - 1], [w1 - 1, 0]
]).reshape(4, 1, 2)

box2 = cv.perspectiveTransform(box1, H)

self.roadImg = cv.polylines(self.roadImg, [np.int32(box2)], True, (0, 255, 0), 4)
```

- 표지판 이미지의 네 귀퉁이 좌표를 도로 영상 상의 대응 좌표로 변환한다.
- `cv.polylines`로 사각형 경계선을 그린다.

---

### 10 매칭 결과 시각화

```python
img_match = np.empty((max(h1, h2), w1 + w2, 3), dtype=np.uint8)
cv.drawMatches(self.signImgs[best], sign_kp, self.roadImg, road_kp,
               good_match, img_match, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv.imshow('Matches and Homography', img_match)
```

- `drawMatches`로 표지판 이미지와 도로 영상 간 매칭된 특징점을 선으로 연결하여 한눈에 확인한다.

---

### 보호구역 알림 + 경고음

```python
self.label.setText(self.signFiles[best][1] + ' 보호구역입니다. 30km로 서행하세요.')
winsound.Beep(3000, 500)
```

- 표지판 종류에 따라 라벨에 안내문을 띄운다.
- `winsound.Beep`로 경고음을 울린다.

---

## 핵심 정리

> 등록된 표지판 이미지와 도로 영상을  
> SIFT + KNN + Ratio Test + Homography 로 비교하여  
> 표지판의 위치를 찾아내고 경계선을 표시하며  
> 안전 알림과 경고음을 출력한다.

| 단계                        | 역할                         |
| ------------------------- | -------------------------- |
| **signFunction()**        | 표지판 모델 이미지 등록              |
| **roadFunction()**        | 도로 영상 선택                   |
| **recognitionFunction()** | SIFT로 표지판 인식, 호모그래피로 위치 표시 |
| **winsound.Beep()**       | 인식 성공 시 경고음 출력             |
| **PyQt5 GUI**             | 버튼 클릭으로 각 기능 실행            |